<a href="https://colab.research.google.com/github/NavaneethaCSR/Aquatic-animal-classification/blob/main/Aquatic_animals_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Function to load images from folder and preprocess for VGG16
def load_images_from_folder(root_folder, img_size=(224, 224)):
    features = []
    labels = []
    class_folders = sorted(os.listdir(root_folder))
    label_map = {class_name: idx for idx, class_name in enumerate(class_folders)}

    for class_folder in class_folders:
        class_path = os.path.join(root_folder, class_folder)
        if os.path.isdir(class_path):
            for filename in os.listdir(class_path):
                img_path = os.path.join(class_path, filename)
                if os.path.isfile(img_path):
                    img = cv2.imread(img_path)
                    if img is not None:
                        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                        img = cv2.resize(img, img_size)
                        img = preprocess_input(img)
                        features.append(img)
                        labels.append(label_map[class_folder])

    return np.array(features), np.array(labels)

# Define paths to your training and testing dataset folders
train_dataset_folder = '/content/drive/MyDrive/Colab Notebooks/Datasets/Aquarium animal classification/train'
test_dataset_folder = '/content/drive/MyDrive/Colab Notebooks/Datasets/Aquarium animal classification/test'

# Load the datasets and preprocess for VGG16
X_train, y_train = load_images_from_folder(train_dataset_folder)
X_test, y_test = load_images_from_folder(test_dataset_folder)

# Data Augmentation for Training
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Augment training data
augmented_images = []
augmented_labels = []

for i in range(len(X_train)):
    img = np.expand_dims(X_train[i], 0)
    it = train_datagen.flow(img, batch_size=1)
    for _ in range(5):  # Generate 5 augmented images per original image
        batch = it.next()
        augmented_images.append(batch[0])
        augmented_labels.append(y_train[i])

X_train_augmented = np.array(augmented_images)
y_train_augmented = np.array(augmented_labels)

# Use a pre-trained VGG16 model for feature extraction
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Extract features using the pre-trained model
X_train_vgg_features = vgg16_model.predict(preprocess_input(X_train_augmented), verbose=1)
X_test_vgg_features = vgg16_model.predict(preprocess_input(X_test), verbose=1)

# Flatten VGG features
X_train_vgg_features = X_train_vgg_features.reshape(X_train_vgg_features.shape[0], -1)
X_test_vgg_features = X_test_vgg_features.reshape(X_test_vgg_features.shape[0], -1)

# Standard scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_vgg_features)
X_test_scaled = scaler.transform(X_test_vgg_features)

# Define individual classifiers
svc_linear = SVC(kernel='linear', C=1, probability=True)
svc_rbf = SVC(kernel='rbf', C=10, gamma=0.01, probability=True)
svc_poly = SVC(kernel='poly', degree=3, C=1, probability=True)
rf_clf = RandomForestClassifier(n_estimators=100)

# Combine classifiers into a voting classifier
voting_clf = VotingClassifier(estimators=[
    ('svc_linear', svc_linear),
    ('svc_rbf', svc_rbf),
    ('svc_poly', svc_poly),
    ('rf', rf_clf)
], voting='soft')

# Train the ensemble classifier
voting_clf.fit(X_train_scaled, y_train_augmented)

# Predicting on the test set
y_pred = voting_clf.predict(X_test_scaled)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

1/1 [==============================] - 9s 9s/step
Accuracy: 0.80
